In [ ]:
import pandas as pd
import warnings
import io
import ipywidgets as widgets
from IPython.display import display, FileLink
import os

warnings.filterwarnings("ignore")

# --- Upload widgets ---
upload_rnsd = widgets.FileUpload(accept='.xlsx', multiple=False, description="Upload RNSD File")
upload_prod = widgets.FileUpload(accept='.xlsx', multiple=False, description="Upload Product File")
upload_xaas = widgets.FileUpload(accept='.xlsx', multiple=False, description="Upload XaaS File")

# --- Output filename widget ---
output_name = widgets.Text(
    value="Final_Report.xlsx",
    description="Output File:",
    style={'description_width': 'initial'},
    placeholder="Enter output file name (e.g., Report.xlsx)"
)

# --- Run button & output area ---
run_button = widgets.Button(description="Run Reports", button_style="success")
output = widgets.Output()

display(upload_rnsd, upload_prod, upload_xaas, output_name, run_button, output)

# --- File helper for ipywidgets 8 ---
def file_to_df(upload_widget, skiprows=0):
    if upload_widget.value:
        # ipywidgets 8 returns dict: {filename: {metadata, content, etc.}}
        for fname, file_info in upload_widget.value.items():
            content = file_info['content']
            if fname.endswith('.xlsx'):
                return pd.read_excel(io.BytesIO(content), skiprows=skiprows)
            else:
                return pd.read_csv(io.BytesIO(content), skiprows=skiprows)
    return None

# --- Format functions ---
def formats(d):
    return pd.DataFrame({
        'MAJOR': d['Part Number'],
        'MINOR': d['Unnamed: 1'],
        'Product': d['Product'],
        'Product Description': d['Product Description'],
        'Price in USD': d['Price in USD'],
        'End of Sale Date': d['End Of Sale Date'],
        'ERP Family': d['ERP Family'],
        'Discount': d['Discount %']
    })

def formats2(d):
    return pd.DataFrame({
        'MAJOR': d['Part Number'],
        'MINOR': d['Unnamed: 1'],
        'Product': d['Product'],
        'Product Description': d['Product Description'],
        'Price in USD': d['Global List Price '],
        'End of Sale Date': d['End Of Sale Date'],
        'ERP Family': d['ERP Family'],
        'Discount': d['Discount %']
    })

# --- Main pipeline ---
def run_reports(b):
    with output:
        output.clear_output()

        rnsd = file_to_df(upload_rnsd, skiprows=0)
        prod = file_to_df(upload_prod, skiprows=1)
        xaas = file_to_df(upload_xaas, skiprows=2)

        if rnsd is None or prod is None or xaas is None:
            print("⚠️ Please upload all three files before running.")
            return

        # Merge logic
        prod = prod.rename(columns={'Unnamed: 0': 'Part Number'})
        prod['Part Number'] = prod['Part Number'].str.upper()
        rnsd['Part Number'] = rnsd['Part Number'].str.upper()
        left1 = pd.merge(prod, rnsd, on='Part Number', how='inner')

        xaas = xaas.rename(columns={'Unnamed: 0': 'Part Number'})
        xaas['Part Number'] = xaas['Part Number'].str.upper()
        left3 = pd.merge(xaas, rnsd, on='Part Number', how='inner')

        left1 = formats(left1)
        left3 = formats2(left3)

        # Combine
        full = pd.merge(left1, left3, how='outer')
        full.drop(columns=['Price in USD', 'End of Sale Date', 'MINOR'], inplace=True)
        full = full[full['ERP Family'].str.upper() != 'SERVICE']

        # Export
        final_name = output_name.value if output_name.value.endswith(".xlsx") else output_name.value + ".xlsx"
        writer = pd.ExcelWriter(final_name, engine='xlsxwriter')
        full.to_excel(writer, index=False, sheet_name="Item Mapping")
        workbook = writer.book
        worksheet = writer.sheets["Item Mapping"]

        (max_row, max_col) = full.shape
        column_settings = [{'header': col} for col in full.columns]
        worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings, 'style': None})
        worksheet.set_column(0, max_col, 12)
        writer.close()

        print(f"✅ Report generated successfully: {final_name}")
        display(full.head())

        # Download link
        if os.path.exists(final_name):
            display(FileLink(final_name, result_html_prefix="📥 Click to download: "))
        else:
            print("⚠️ Error: File not found after saving.")

run_button.on_click(run_reports)